# LCM Miner tool

In [2]:
import pandas as pd 
import numpy as np
from datetime import date,datetime
import csv
import re
from multiprocessing import Pool
from functools import partial
from sklearn import preprocessing
from IPython.core import display as ICD


### 1. Une fonction qui prend les arguments suivants:
jeu de données (contenant timestamp)
fréquence de temps choisie (année, mois, jour) et qui retourne le jeu de données partitionné selon la granularité temporelle sous la forme de plusieurs fichiers, un par couple (partition, période de temps)

In [3]:
def split_dataset(input_file,frequency,dataset_filter):
    """Split initial dataset to multiple files according to frequency
    
    Yields
    ------
    str
        names of partitions files
    """
    df = pd.read_csv(input_file)
    df.query(dataset_filter,inplace=True)
    df["timestamp"] = pd.to_datetime(df["timestamp"],unit="s").dt.to_period(frequency)
    df.set_index(["timestamp"],inplace=True)
    df.movieId = df.movieId.astype(str)
    
    for period,period_df in df.groupby(pd.Grouper(freq = frequency)):
        output = period_df.groupby("userId").movieId.apply(lambda x: " ".join(x)).to_frame().reset_index()
        output.movieId = output.userId.astype(str)+" "+ output.movieId
        output.drop(["userId"],inplace=True,axis=1)
        output.to_csv(str(period),header=None,index=None)
        output.movieId.apply(lambda x: x.split(" ")[0]).to_frame().astype(str).to_csv("permut"+str(period),header=None,index=False)        
        yield period
        

### 2. Une fonction qui prend les arguments suivants:
- un couple (partition, période de temps)
- la valeur du support
<br>Cette fonction fait appel à pmr.pylcm et retourne des groupes fréquents selon leur support
Group description:[set of items] (support) Group content:[set of users]

In [4]:
def format_output(results,period):
    """Format lcm output to:
    Group description:[set of items] (support) Group content:[Sorted set of users] 
    """
    item_sets,supports,groups,periods = [],[],[],[]
    for i,j in zip(results[3::2],results[4::2]):
        if '(' not in i:
            break
        *items,support = re.findall("([0-9]+)+",i)
        items = ','.join(e for e in items)
        group = j[1:]
        item_sets.append(items)
        supports.append(support)
        periods.append(period)
        groups.append(group)
    return item_sets,supports,groups,periods

def run_lcm(input_file=None,support=None,groups_size=[5,100],output=""):
    """ Runs LCM  and return the  result formated with format_output"""
    
    result = !./lcm CfI -l {groups_size[0]} -u {groups_size[1]} {str(input_file)} {support} {str(input_file)+output}
    if "there is no frequent item" in str(result):
        print("No itemset",input_file)
        return ''
    return str(input_file)+output

### 3. Une fonction qui prend en argument:
Group description:[set of items] (support) Group content:[set of users]
et qui retourne pour chaque user, l’ensemble des groupes auxquels il appartient sous la forme:
(user_id, group_id, période de temps)


In [5]:
def users_groups(df,output="result.csv"):
    # split each user from the group to a new line : [[user_id,group,period],]
    res = pd.DataFrame(df.users_ids.str.split(",").tolist(),index=[df.period,df.group]).stack()
    res = res.reset_index([0,"group","period"])
    res.columns = ["period","group_id","user_id"]
    res = res[['user_id','group_id','period']]
    return res

### Mining function 

In [6]:
def singlethread_lcm(input_file='data/ml-latest-small/ratings.csv',frequency="Y",support=20 ):
    """ Run LCM with a single thread and output the result to output file
    Output structure :
        user_id,group_id,period
    """
    output = []
    for partition in split_dataset(input_file,frequency):
        # run LCM and get the items, supports and groups of each frequent itemset found 
        item_sets,supports,groups = run_lcm(partition,frequency,support)
        # get the affectation of each user from the frequence itemsets groups 
        users_affectation = users_groups(groups=groups,period=str(partition))
        output+=users_affectation
    return output

def multithread_lcm(input_file,frequency,support,groups_size,dataset_filter):
    f = partial(run_lcm,support=support,groups_size=groups_size)
    p = Pool(8)
    res  = p.map(f,split_dataset(input_file,frequency,dataset_filter))
    p.close()
    p.join()
    return res

def permut(x,data):
    output= ""
    for i in x.split(' '):
        if i is "":
            continue
        output+= str(data[0][int(i)])+' '
    return output
outputs = []



def form_group(x,permutations):
    output = ''
    for i in x.split(" "):
        if i is not "":
            output+=str(permutations[0][int(i)])+","     
    return output[:-1]

def combine_outputs(files_names):
    """
    Output : Dataframe,Dataframe
        content of files in files_names in one dataframe
        content of permutations files associated in one dataframe 
    """
    combined_csv = pd.DataFrame()
    permutations = pd.DataFrame()

    for i in files_names:
        permut_file = "permut"+str(i)
        try:# when l is too small even if lcm find results, file is created by empty
            df = pd.read_csv(i,header=None)
        except:
            continue
        df["period"] = i 
        combined_csv= pd.concat([combined_csv,df])
        df = pd.read_csv(permut_file ,header=None)
        df["period"] = i 
        permutations= pd.concat([permutations,df])
    return combined_csv,permutations

def reformat_output(output):
    """This function reformat outputs of lcm :
    From multiple files having structure : (support, itemsets) in peer rows and (#trasaction, period)in odd rows 
    each file  is associated with a permutation file  ('permut'+file_name) in which there is (#trasaction,user_id)
    1- concatenate all output to one dataframe "df" , and all permutations to one dataframe "permutations"
    2- reformat df structure to (itemset,support,#transactions,period)
    3- replace each #transaction with it's user_id
    Input :
        [files_names] 
    Output: Dataframe:
        itemset,support,users_ids
    """
    df,permutations = combine_outputs(output)
    df.reset_index(drop=True,inplace=True)
    # join odd and pair rows to same row
    a = df[0][0::2].reset_index()+df[0][1::2].reset_index()
    # split row to three columns :(itemsets,support,users_ids)
    a = a[0].str.split("(\([0-9]+\))",expand=True).replace(['\(','\)']," ",regex=True)
    
    df = pd.concat([df["period"][0::2].reset_index(drop=True),a],axis=1)
    df.columns=["period","itemset","support","users_ids"]
    
    for (period,i) in df.groupby("period"):
        df["users_ids"][i.index]= i["users_ids"].apply(lambda x:form_group(x,permutations[permutations.period==period]))
    return df 
def encode_groups(df):
    """Label encoding from column users_ids in dataframe input"""
    le = preprocessing.LabelEncoder()
    df["group"]= le.fit_transform(df["users_ids"])

# Run the code 

### LCM minning in multithread 
#### Output : file name  of each period having results

In [7]:
input_file = "ratings.csv"
frequency = "M" # [num| ][D|M|Y] ex Y,M, 2M,  
support = 2
groups_size = [2,100]
dataset_filter = "rating>3"
output_file = f'{frequency}-{support}-[{groups_size[0]}-{groups_size[1]}]-{dataset_filter}-groups.dat'
def linear_closed_itemset_miner(input_file,frequency,support,groups_size,output_file,dataset_filter):
    output = multithread_lcm(input_file,frequency,support,groups_size,dataset_filter)
    output = [i for i in  output if i is not ""]
    df = reformat_output(output)
    encode_groups(df)
    !rm permut*
    !rm 1*
    !rm 2*
    users_groups(df).to_csv(output_file)
    return df
# df = linear_closed_itemset_miner(input_file,frequency,support,groups_size,output_file,dataset_filter)
# df.group.value_counts().to_frame().head()

'M-2-[2-100]-groups.dat' => 

In [10]:
for frequency in ["Y","6M","3M","2M","M"]:
    for support in [2 ,5 ,10, 15, 20, 30, 50 ]:
        for groups in [[2,5],[2,10],[2,20],[20,50]]:
            for dataset_filter in ["rating>3","rating>4","rating<3","rating<2","2<rating<4","rating==5"]:
                output_file = f'{frequency}-{support}-[{groups_size[0]}-{groups_size[1]}]-{dataset_filter}-groups.dat'
                try:
                    df = linear_closed_itemset_miner(input_file,frequency,support,groups,output_file,dataset_filter)
                    print(output_file)
                    ICD.display(df.group.value_counts().to_frame().head())
                except Exception as e:
                    print(input_file,frequency,support,groups_size,output_file,dataset_filter)
                    print(e)

Y-2-[2-100]-rating>3-groups.dat


,group
27979,5
51041,4
51080,4
21832,4
51032,3


Y-2-[2-100]-rating>4-groups.dat


,group
5594,3
3310,2
2522,2
380,2
2169,2


Y-2-[2-100]-rating<3-groups.dat


,group
797,4
798,4
560,3
238,2
431,2


No itemset 2013
Y-2-[2-100]-rating<2-groups.dat


,group
105,1
26,1
28,1
29,1
30,1


KeyboardInterrupt: 

In [46]:
files = !ls *.dat
for i in files:
    ICD.display(pd.read_csv(i).groupby[""])
    

,Unnamed: 0,user_id,group_id,period
0,0,102,0,1996-08
1,1,170,0,1996-08
2,2,176,0,1996-08
3,3,179,0,1996-08
4,4,229,0,1996-08
5,5,235,0,1996-08
6,6,385,0,1996-08
7,7,394,0,1996-08
8,8,498,0,1996-08
9,9,541,0,1996-08


,Unnamed: 0,user_id,group_id,period
0,0,56,199,1996-08
1,1,102,199,1996-08
2,2,109,199,1996-08
3,3,142,199,1996-08
4,4,179,199,1996-08
5,5,229,199,1996-08
6,6,235,199,1996-08
7,7,385,199,1996-08
8,8,394,199,1996-08
9,9,404,199,1996-08


,Unnamed: 0,user_id,group_id,period
0,0,38,0,1996-08
1,1,109,0,1996-08
2,2,176,0,1996-08
3,3,179,0,1996-08
4,4,229,0,1996-08
5,5,447,0,1996-08
6,6,486,0,1996-08
7,7,498,0,1996-08
8,8,512,0,1996-08
9,9,574,0,1996-08


,Unnamed: 0,user_id,group_id,period
0,0,38,0,1996-08
1,1,109,0,1996-08
2,2,176,0,1996-08
3,3,179,0,1996-08
4,4,229,0,1996-08
5,5,447,0,1996-08
6,6,486,0,1996-08
7,7,498,0,1996-08
8,8,512,0,1996-08
9,9,574,0,1996-08


,Unnamed: 0,user_id,group_id,period
0,0,56,29,1996-08
1,1,102,29,1996-08
2,2,109,29,1996-08
3,3,142,29,1996-08
4,4,179,29,1996-08
5,5,229,29,1996-08
6,6,235,29,1996-08
7,7,385,29,1996-08
8,8,394,29,1996-08
9,9,404,29,1996-08


...

,Unnamed: 0,user_id,group_id,period
0,0,5,2653,1996
1,1,6,2653,1996
2,2,37,2653,1996
3,3,58,2653,1996
4,4,81,2653,1996
5,5,94,2653,1996
6,6,102,2653,1996
7,7,117,2653,1996
8,8,145,2653,1996
9,9,170,2653,1996


,Unnamed: 0,user_id,group_id,period
0,0,81,21,1996
1,1,126,21,1996
2,2,235,21,1996
3,3,314,21,1996
4,4,394,21,1996
5,5,81,20,1996
6,6,117,20,1996
7,7,126,20,1996
8,8,136,20,1996
9,9,411,20,1996


,Unnamed: 0,user_id,group_id,period
0,0,6,40084,1996
1,1,37,40084,1996
2,2,43,40084,1996
3,3,56,40084,1996
4,4,58,40084,1996
5,5,102,40084,1996
6,6,109,40084,1996
7,7,117,40084,1996
8,8,126,40084,1996
9,9,133,40084,1996


,Unnamed: 0,user_id,group_id,period
0,0,6,770,1996
1,1,38,770,1996
2,2,58,770,1996
3,3,94,770,1996
4,4,107,770,1996
5,5,109,770,1996
6,6,117,770,1996
7,7,130,770,1996
8,8,176,770,1996
9,9,179,770,1996


,Unnamed: 0,user_id,group_id,period
0,0,6,462,1996
1,1,38,462,1996
2,2,58,462,1996
3,3,94,462,1996
4,4,107,462,1996
5,5,109,462,1996
6,6,117,462,1996
7,7,130,462,1996
8,8,176,462,1996
9,9,179,462,1996
